<a href="https://colab.research.google.com/github/cxw1219/Lion/blob/main/Lion%F0%9F%A6%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [64]:
# Setup and imports
!pip install torch pandas numpy scikit-learn matplotlib seaborn
import torch, pandas as pd, numpy as np
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler

In [65]:
# Data loading functions
def load_data(file_path):
    return pd.read_csv(file_path, parse_dates=['timestamp'])

def calculate_usdx(forex_data):
    weights = {
        'EUR': -0.576, 'JPY': 0.136, 'GBP': -0.119,
        'CAD': 0.091, 'SEK': 0.042, 'CHF': 0.036
    }
    usdx = 50.14348112
    for curr, weight in weights.items():
        if curr == 'EUR':
            pair = forex_data['EUR_USD']
        elif curr == 'GBP':
            pair = forex_data['GBP_USD']
        else:
            pair = forex_data[f'USD_{curr}']
        usdx *= pair['close'] ** weight
    return pd.Series(usdx, index=pair.index)

In [66]:
# Feature engineering
def create_features(df):
    return pd.DataFrame({
        'close': df['close'],
        'returns': df['close'].pct_change(fill_method=None),
        'volatility': df['close'].pct_change(fill_method=None).rolling(20).std()
    })

In [67]:
class GoldPredictor(torch.nn.Module):
    def __init__(self, input_dim=9):
        super().__init__()
        self.embed = torch.nn.Linear(input_dim, 8)  # Project to dimension divisible by 2
        encoder_layer = torch.nn.TransformerEncoderLayer(
            d_model=8,  # Now divisible by 2
            nhead=2,
            dim_feedforward=256,
            batch_first=True
        )
        self.encoder = torch.nn.TransformerEncoder(encoder_layer, num_layers=3)
        self.decoder = torch.nn.Linear(8, 12)
        self.softplus = torch.nn.Softplus()

    def forward(self, x):
        x = x.unsqueeze(0) if x.dim() == 2 else x
        x = self.embed(x)
        x = self.encoder(x)
        x = self.decoder(x[:, -1, :])
        mean, scale = x.chunk(2, dim=-1)
        return mean, self.softplus(scale)

In [68]:
# Data processing pipeline with chunking
def prepare_data(sequence_length=144, chunk_size=1000):
    data_path = '/content/drive/MyDrive/UpsharesDownshares/ONN/Data'

    def load_in_chunks(filepath):
        chunks = pd.read_csv(filepath, dtype={'timestamp': 'object'}, chunksize=chunk_size)
        df = pd.concat(chunks)
        df['timestamp'] = pd.to_datetime(df['timestamp'], format='%Y-%m-%d %H:%M:%S%z')
        return df.set_index('timestamp').sort_index()

    # Load gold data
    gold_data = create_features(load_in_chunks(f'{data_path}/XAU_USD.csv'))

    # Load bond data
    bond_data = create_features(load_in_chunks(f'{data_path}/USB02Y_USD.csv'))

    # Process forex data
    forex_files = ['EUR_USD.csv', 'USD_JPY.csv', 'GBP_USD.csv',
                   'USD_CAD.csv', 'USD_SEK.csv', 'USD_CHF.csv']

    forex_data = {}
    for f in forex_files:
        forex_data[f.split('.')[0]] = load_in_chunks(f'{data_path}/{f}')

    usdx = create_features(pd.DataFrame({'close': calculate_usdx(forex_data)}))

    # Combine and align on timestamps
    data = pd.concat([gold_data, usdx, bond_data], axis=1).dropna()

    # Split and scale
    train_mask = data.index <= '2020-12'
    scaler = StandardScaler().fit(data[train_mask])
    scaled_data = scaler.transform(data)

    return scaled_data, scaler

In [69]:
# Chunked sequence training
def train_model(model, train_data, epochs=100, seq_length=144, chunk_size=1000):
    torch.cuda.empty_cache()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

    # Create sequences
    n_sequences = len(train_data) - seq_length - 6
    n_chunks = n_sequences // chunk_size + 1

    best_loss = float('inf')
    patience = 5
    no_improve = 0

    for epoch in range(epochs):
        total_loss = 0

        for chunk in range(n_chunks):
            start_idx = chunk * chunk_size
            end_idx = min(start_idx + chunk_size, n_sequences)

            sequences = torch.from_numpy(np.array([
                train_data[i:i+seq_length]
                for i in range(start_idx, end_idx)
            ])).float().to(device)

            targets = torch.from_numpy(np.array([
                train_data[i+seq_length:i+seq_length+6, 0]
                for i in range(start_idx, end_idx)
            ])).float().to(device)

            model.train()
            mean, std = model(sequences)
            loss = -torch.distributions.Normal(mean, std).log_prob(targets).mean()

            optimizer.zero_grad(set_to_none=True)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

            # Free memory
            del sequences
            del targets
            torch.cuda.empty_cache()

            print(f'\rEpoch {epoch+1}/{epochs} Chunk {chunk+1}/{n_chunks} Loss: {loss:.4f} GPU: {torch.cuda.memory_allocated()/1e9:.1f}GB', end='')

        avg_loss = total_loss / n_chunks
        print(f'\nEpoch {epoch+1} Average Loss: {avg_loss:.4f}')

        if abs(avg_loss - best_loss) < 0.0001:
            no_improve += 1
            if no_improve >= patience:
                print("Early stopping - loss converged")
                break
        else:
            no_improve = 0
            if avg_loss < best_loss:
                best_loss = avg_loss

In [70]:
# Inference functions
def predict(model, recent_data, scaler):
    model.eval()
    with torch.no_grad():
        mean, std = model(recent_data)
        mean_cpu = mean.cpu().numpy()
        std_cpu = std.cpu().numpy()

        # Only unscale the gold price (first feature)
        preds = mean_cpu * scaler.scale_[0] + scaler.mean_[0]
        conf = std_cpu * scaler.scale_[0]

        return preds, conf

def display_predictions(predictions, confidence):
    # Create timestamps for predictions only (not including current time)
    times = pd.date_range(start=pd.Timestamp.now(), periods=6, freq='10min')
    return pd.DataFrame({
        'Price': predictions,
        'Lower_CI': predictions - 1.96 * confidence,
        'Upper_CI': predictions + 1.96 * confidence
    }, index=times)

In [71]:
# Main execution
if __name__ == "__main__":
    print("Starting pipeline...")

    print("Checking GPU...", end=' ')
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using {device}")

    print("Loading data...", end=' ')
    data, scaler = prepare_data()
    print(f"Loaded {len(data)} samples")

    print("Splitting data...", end=' ')
    split_idx = int(len(data) * 0.8)
    train_data = data[:split_idx]
    test_data = data[split_idx:]
    print(f"Train: {len(train_data)}, Test: {len(test_data)}")

    print("Initializing model...", end=' ')
    model = GoldPredictor().to(device)
    print("Done")

    print("\nStarting training...")
    train_model(model, train_data)

    print("\nGenerating predictions...")
    latest_data = torch.FloatTensor(data[-144:]).unsqueeze(0).to(device)
    preds, conf = predict(model, latest_data, scaler)
    print("\nPredictions:")
    print(display_predictions(preds[0], conf[0]))

Starting pipeline...
Checking GPU... Using cuda
Loading data... Loaded 320011 samples
Splitting data... Train: 256008, Test: 64003
Initializing model... Done

Starting training...
Epoch 1/100 Chunk 256/256 Loss: 3.8705 GPU: 0.0GB
Epoch 1 Average Loss: 2.7753
Epoch 2/100 Chunk 256/256 Loss: 3.3993 GPU: 0.0GB
Epoch 2 Average Loss: 2.2609
Epoch 3/100 Chunk 256/256 Loss: 3.0607 GPU: 0.0GB
Epoch 3 Average Loss: 2.1199
Epoch 4/100 Chunk 256/256 Loss: 2.8096 GPU: 0.0GB
Epoch 4 Average Loss: 2.0084
Epoch 5/100 Chunk 256/256 Loss: 2.5952 GPU: 0.0GB
Epoch 5 Average Loss: 1.8694
Epoch 6/100 Chunk 256/256 Loss: 2.4051 GPU: 0.0GB
Epoch 6 Average Loss: 1.6924
Epoch 7/100 Chunk 256/256 Loss: 2.2521 GPU: 0.0GB
Epoch 7 Average Loss: 1.5037
Epoch 8/100 Chunk 256/256 Loss: 2.1278 GPU: 0.0GB
Epoch 8 Average Loss: 1.3991
Epoch 9/100 Chunk 256/256 Loss: 2.0273 GPU: 0.0GB
Epoch 9 Average Loss: 1.3309
Epoch 10/100 Chunk 256/256 Loss: 1.9454 GPU: 0.0GB
Epoch 10 Average Loss: 1.2777
Epoch 11/100 Chunk 256/256 L

<ipython-input-70-b551d43795d5>:16: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  times = pd.date_range(start=pd.Timestamp.now(), periods=7, freq='10T')


ValueError: Length of values (6) does not match length of index (7)

In [ ]:
# Evaluation metrics
def evaluate_model(model, test_data, test_targets):
    predictions, confidence = predict(model, test_data)

    metrics = {
        'RMSE': np.sqrt(((predictions - test_targets) ** 2).mean()),
        'MAE': np.abs(predictions - test_targets).mean(),
        'MAPE': np.abs((predictions - test_targets) / test_targets).mean() * 100,
        'Direction': (np.sign(predictions[1:] - predictions[:-1]) ==
                     np.sign(test_targets[1:] - test_targets[:-1])).mean()
    }

    return metrics

In [ ]:
# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

def plot_results(model, test_data, test_targets):
    preds, conf = predict(model, test_data)

    plt.figure(figsize=(12, 6))
    plt.plot(test_targets, label='Actual')
    plt.plot(preds, label='Predicted')
    plt.fill_between(range(len(preds)),
                    preds - 1.96 * conf,
                    preds + 1.96 * conf,
                    alpha=0.3)
    plt.title('Gold Price Predictions with 95% CI')
    plt.legend()
    plt.show()